## Mapping **U.S. News & World Report**'s
## [_125 Best Places to Live in the USA_](https://realestate.usnews.com/places/rankings/best-places-to-live)

If you would like to review their [methodology:](https://realestate.usnews.com/places/methodology)
>U.S. News & World Report's Best Places to Live rankings are intended to help readers make the most informed decision when choosing where to settle down. The metro areas included in the rankings are evaluated using data from trusted sources like the United States Census Bureau, the Federal Bureau of Investigation, the Department of Labor and U.S. News' own internal resources. This data was categorized into the five indexes listed below and then evaluated using a methodology determined by Americans' preferences. The percent weighting for each index follows the answers from a public survey in which people from across the country voted for what they believed was the most important thing to consider when thinking about moving. 

### Let's get started:

In [1]:
from ipyleaflet import Map, GeoData, basemaps, LayersControl, FullScreenControl, DrawControl, Heatmap, CircleMarker, ScaleControl, ZoomControl, Marker, Popup
from ipywidgets import HTML

import geopandas
import pandas as pd
import json
import random
from random import uniform


/anaconda3/lib/python3.6/site-packages/distributed/config.py:63: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config.update(yaml.load(text) or {})


#### To ensure we are up and running, let's plot the example given from [ipyleaflet's GeoData](https://ipyleaflet.readthedocs.io/en/latest/api_reference/geodata.html#example):

In [2]:
countries = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
rivers = geopandas.read_file("https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/physical/ne_10m_rivers_lake_centerlines.zip")

m = Map(center=(52.3,8.0), zoom = 3, basemap= basemaps.Esri.WorldTopoMap)

geo_data = GeoData(geo_dataframe = countries,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Countries')

rivers_data = GeoData(geo_dataframe = rivers,
                   style={'color': 'purple', 'opacity':3, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Rivers')

m.add_layer(rivers_data)
m.add_layer(geo_data)
m.add_control(LayersControl())

m

Map(center=[52.3, 8.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

Now that we have confirmed we can plot the example, let's try a small map of our own:

In [3]:
Map(center=[ 40, -90], zoom = 3)

Map(center=[40, -90], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

### Import Data
Next, let's import some the US News Data, along with some City coordinates.
This is done locally for now, with sources. We could work on pulling these from the web at a later date.

In [4]:
BASE = "/Users/kl023465/OneDrive - Rockhurst University/6313 GIS/Final/data/"

#### Data Sources:
* [best-places-to-live](https://realestate.usnews.com/places/rankings/best-places-to-live)
* [1000-largest-us-cities-by-population-with-geographic-coordinates](https://public.opendatasoft.com/explore/embed/dataset/1000-largest-us-cities-by-population-with-geographic-coordinates/)
* [naturalearthdata features](https://www.naturalearthdata.com/features/)

In [5]:
# Best Places To Live
USNEWS = pd.read_csv(BASE + "cities.csv", sep = ";")

# Some Coordinate Data
cities = geopandas.read_file(BASE + "1000-largest-us-cities-by-population-with-geographic-coordinates.geojson")

# Natural Earth Features
states = geopandas.read_file(BASE + "ne_50m_admin_1_states_provinces/ne_50m_admin_1_states_provinces.shp")
rivers = geopandas.read_file(BASE + "ne_50m_rivers_lake_centerlines/ne_50m_rivers_lake_centerlines.shp")
lakes = geopandas.read_file(BASE + "ne_50m_lakes/ne_50m_lakes.shp")
airport = geopandas.read_file(BASE + "ne_50m_airports/ne_50m_airports.shp")
coastline = geopandas.read_file(BASE + "ne_50m_coastline/ne_50m_coastline.shp")


Let's preview what the data set looks like:

In [6]:
USNEWS.head()

,City,State,US News Rank,Text,Overall Score,Quality of Life,Value
0,Austin,TX,1,"People are drawn to the Texas capital's music,...",7.6,7.3,6.7
1,Denver,CO,2,Founded in the mid-1800s as a mining hub durin...,7.4,7.0,6.7
2,Colorado Springs,CO,3,Colorado Springs might not jump off the map as...,7.4,7.0,6.6
3,Fayetteville,AR,4,Located in a region that's experiencing drasti...,7.3,7.5,8.2
4,Des Moines,IA,5,The capital of this so-called flyover state ma...,7.3,7.0,8.4


In [7]:
USNEWS.count()

City               125
State              125
US News Rank       125
Text               125
Overall Score      125
Quality of Life    125
Value              125
dtype: int64

In [8]:
cities.head()

,city,rank,state,growth_from_2000_to_2013,population,geometry
0,Marysville,552,Washington,115.7,63269,POINT (-122.17708 48.05176)
1,Perris,466,California,98.7,72326,POINT (-117.22865 33.78252)
2,Cleveland,48,Ohio,-18.1,390113,POINT (-81.69436 41.49932)
3,Worcester,129,Massachusetts,5.8,182544,POINT (-71.80229 42.26259)
4,Columbia,192,South Carolina,11.7,133358,POINT (-81.03481 34.00071)


Check the CRS being used by the data set:

In [9]:
cities.crs

{'init': 'epsg:4326'}

### Data Manipulation:

In [10]:
#From: https://gist.githubusercontent.com/tlancon/9794920a0c3a9990279de704f936050c/raw/dd7f7db501af25287a0daeaf3d90185a695d85ea/us_states_dict.py
us_states = {
  'Alabama':'AL','Alaska':'AK','Arizona':'AZ','Arkansas':'AR','California':'CA','Colorado':'CO','Connecticut':'CT','Delaware':'DE','District of Columbia':'DC','Florida':'FL','Georgia':'GA','Hawaii':'HI','Idaho':'ID','Illinois':'IL','Indiana':'IN','Iowa':'IA','Kansas':'KS','Kentucky':'KY','Louisiana':'LA','Maine':'ME','Maryland':'MD','Massachusetts':'MA','Michigan':'MI','Minnesota':'MN','Mississippi':'MS','Missouri':'MO','Montana':'MT','Nebraska':'NE','Nevada':'NV','New Hampshire':'NH','New Jersey':'NJ','New Mexico':'NM','New York':'NY','North Carolina':'NC','North Dakota':'ND','Ohio':'OH','Oklahoma':'OK','Oregon':'OR','Pennsylvania':'PA','Rhode Island':'RI','South Carolina':'SC','South Dakota':'SD','Tennessee':'TN','Texas':'TX','Utah':'UT','Vermont':'VT','Virginia':'VA','Washington':'WA','West Virginia':'WV','Wisconsin':'WI','Wyoming':'WY'
}

In [11]:
cities["state_code"] = cities["state"].map(us_states)

In [12]:
cities["city_key"] = cities["city"] + "-" + cities["state_code"]
cities["city_key"]

0         Marysville-WA
1             Perris-CA
2          Cleveland-OH
3          Worcester-MA
4           Columbia-SC
             ...       
995    Santa Barbara-CA
996          Addison-IL
997       Rio Rancho-NM
998           Normal-IL
999       Romeoville-IL
Name: city_key, Length: 1000, dtype: object

In [13]:
cities.head()

,city,rank,state,growth_from_2000_to_2013,population,geometry,state_code,city_key
0,Marysville,552,Washington,115.7,63269,POINT (-122.17708 48.05176),WA,Marysville-WA
1,Perris,466,California,98.7,72326,POINT (-117.22865 33.78252),CA,Perris-CA
2,Cleveland,48,Ohio,-18.1,390113,POINT (-81.69436 41.49932),OH,Cleveland-OH
3,Worcester,129,Massachusetts,5.8,182544,POINT (-71.80229 42.26259),MA,Worcester-MA
4,Columbia,192,South Carolina,11.7,133358,POINT (-81.03481 34.00071),SC,Columbia-SC


In [14]:
USNEWS["city_key"] = USNEWS["City"] + "-"+ USNEWS["State"]

In [15]:
USNEWS.head()

,City,State,US News Rank,Text,Overall Score,Quality of Life,Value,city_key
0,Austin,TX,1,"People are drawn to the Texas capital's music,...",7.6,7.3,6.7,Austin-TX
1,Denver,CO,2,Founded in the mid-1800s as a mining hub durin...,7.4,7.0,6.7,Denver-CO
2,Colorado Springs,CO,3,Colorado Springs might not jump off the map as...,7.4,7.0,6.6,Colorado Springs-CO
3,Fayetteville,AR,4,Located in a region that's experiencing drasti...,7.3,7.5,8.2,Fayetteville-AR
4,Des Moines,IA,5,The capital of this so-called flyover state ma...,7.3,7.0,8.4,Des Moines-IA


In [16]:
cities_joined = cities.merge(USNEWS, how = 'right', on=['city_key'])

In [17]:
type(cities_joined)

geopandas.geodataframe.GeoDataFrame

In [18]:
type(USNEWS)

pandas.core.frame.DataFrame

In [19]:
cities_joined

,city,rank,state,growth_from_2000_to_2013,population,geometry,state_code,city_key,City,State,US News Rank,Text,Overall Score,Quality of Life,Value
0,Cleveland,48.0,Ohio,-18.1,390113.0,POINT (-81.69436 41.49932),OH,Cleveland-OH,Cleveland,OH,85,Once a haven for immigrants and a powerhouse o...,6.3,6.1,7.4
1,Worcester,129.0,Massachusetts,5.8,182544.0,POINT (-71.80229 42.26259),MA,Worcester-MA,Worcester,MA,62,A vibrant and diverse community nicknamed The ...,6.5,7.0,6.5
2,Columbia,192.0,South Carolina,11.7,133358.0,POINT (-81.03481 34.00071),SC,Columbia-SC,Columbia,SC,67,"Columbia, South Carolina, is a traditionally S...",6.5,6.1,7.2
3,Lafayette,214.0,Louisiana,11.0,124276.0,POINT (-92.01984 30.22409),LA,Lafayette-LA,Lafayette,LA,96,"In 2014, Lafayette was named the happiest city...",6.2,5.4,8.1
4,Boise City,98.0,Idaho,9.5,214237.0,POINT (-116.21461 43.61871),ID,Boise City-ID,Boise City,ID,17,Boise is a recreationalist's paradise. If you ...,7.1,7.2,7.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,Santa Barbara,339.0,California,0.9,90412.0,POINT (-119.69819 34.42083),CA,Santa Barbara-CA,Santa Barbara,CA,73,"It has been described as the ""American Riviera...",6.5,7.8,3.7
121,NaN,NaN,NaN,NaN,NaN,None,NaN,Lexington-KY,Lexington,KY,29,Lexington blends influences from the South and...,6.9,6.9,7.7
122,NaN,NaN,NaN,NaN,NaN,None,NaN,Augusta-GA,Augusta,GA,72,"Located near the South Carolina border, August...",6.5,6.4,7.1
123,NaN,NaN,NaN,NaN,NaN,None,NaN,Myrtle Beach-SC,Myrtle Beach,SC,75,Although it's better known to outsiders as a v...,6.4,6.4,5.6


In [20]:
cities_joined["geometry"].crs

{'init': 'epsg:4326'}

In [21]:
type(cities_joined["geometry"])

geopandas.geoseries.GeoSeries

In [22]:
cities_joined["geometry"].head()

0     POINT (-81.69436 41.49932)
1     POINT (-71.80229 42.26259)
2     POINT (-81.03481 34.00071)
3     POINT (-92.01984 30.22409)
4    POINT (-116.21461 43.61871)
Name: geometry, dtype: geometry

###  Calculate Minimum Distance to Lakes, Rivers, Coast

In [23]:
# Have to drop 4 straglers at this time: Agusta, Lexington, Myrtle Beach, & San Juan
cities_joined = cities_joined[:121]

#### Lets Give a Scaled Value for features near our points:

In [24]:
cities_joined["geometry"][0].distance(lakes.geometry[4])

0.022174903909946996

In [25]:
type(lakes)

geopandas.geodataframe.GeoDataFrame

#### Calculate Minimum Distance to Features

In [26]:
# https://stackoverflow.com/questions/30740046/calculate-distance-to-nearest-feature-with-geopandas?rq=1
def min_distance(point, lines):
    return lines.distance(point).min()

cities_joined['min_dist_to_rivers'] = cities_joined.geometry.apply(min_distance, args=(rivers,))
cities_joined['min_dist_to_coasts'] = cities_joined.geometry.apply(min_distance, args=(coastline,))
cities_joined['min_dist_to_lakes'] = cities_joined.geometry.apply(min_distance, args=(lakes,))


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [27]:
cities_joined.head()

,city,rank,state,growth_from_2000_to_2013,population,geometry,state_code,city_key,City,State,US News Rank,Text,Overall Score,Quality of Life,Value,min_dist_to_rivers,min_dist_to_coasts,min_dist_to_lakes
0,Cleveland,48.0,Ohio,-18.1,390113.0,POINT (-81.69436 41.49932),OH,Cleveland-OH,Cleveland,OH,85,Once a haven for immigrants and a powerhouse o...,6.3,6.1,7.4,1.385577,5.297741,0.022175
1,Worcester,129.0,Massachusetts,5.8,182544.0,POINT (-71.80229 42.26259),MA,Worcester-MA,Worcester,MA,62,A vibrant and diverse community nicknamed The ...,6.5,7.0,6.5,1.962580,0.623077,1.373716
2,Columbia,192.0,South Carolina,11.7,133358.0,POINT (-81.03481 34.00071),SC,Columbia-SC,Columbia,SC,67,"Columbia, South Carolina, is a traditionally S...",6.5,6.1,7.2,1.057882,1.538227,6.822463
3,Lafayette,214.0,Louisiana,11.0,124276.0,POINT (-92.01984 30.22409),LA,Lafayette-LA,Lafayette,LA,96,"In 2014, Lafayette was named the happiest city...",6.2,5.4,8.1,0.723258,0.407954,1.439853
4,Boise City,98.0,Idaho,9.5,214237.0,POINT (-116.21461 43.61871),ID,Boise City-ID,Boise City,ID,17,Boise is a recreationalist's paradise. If you ...,7.1,7.2,7.1,0.386370,7.195404,2.419178


#### Make an object for each feature::

In [28]:
x_es = cities_joined.geometry.x.tolist()
y_es = cities_joined.geometry.y.tolist()


In [29]:
cities_joined['rivers_scaled'] = (1/cities_joined['min_dist_to_rivers']) * 1000
cities_joined['coasts_scaled'] = (1/cities_joined['min_dist_to_coasts']) * 1000
cities_joined['lakes_scaled'] = (1/cities_joined['min_dist_to_lakes']) * 1000


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [30]:
cities_joined.head()

,city,rank,state,growth_from_2000_to_2013,population,geometry,state_code,city_key,City,State,...,Text,Overall Score,Quality of Life,Value,min_dist_to_rivers,min_dist_to_coasts,min_dist_to_lakes,rivers_scaled,coasts_scaled,lakes_scaled
0,Cleveland,48.0,Ohio,-18.1,390113.0,POINT (-81.69436 41.49932),OH,Cleveland-OH,Cleveland,OH,...,Once a haven for immigrants and a powerhouse o...,6.3,6.1,7.4,1.385577,5.297741,0.022175,721.721102,188.759703,45096.024049
1,Worcester,129.0,Massachusetts,5.8,182544.0,POINT (-71.80229 42.26259),MA,Worcester-MA,Worcester,MA,...,A vibrant and diverse community nicknamed The ...,6.5,7.0,6.5,1.962580,0.623077,1.373716,509.533275,1604.939242,727.952364
2,Columbia,192.0,South Carolina,11.7,133358.0,POINT (-81.03481 34.00071),SC,Columbia-SC,Columbia,SC,...,"Columbia, South Carolina, is a traditionally S...",6.5,6.1,7.2,1.057882,1.538227,6.822463,945.285376,650.099250,146.574634
3,Lafayette,214.0,Louisiana,11.0,124276.0,POINT (-92.01984 30.22409),LA,Lafayette-LA,Lafayette,LA,...,"In 2014, Lafayette was named the happiest city...",6.2,5.4,8.1,0.723258,0.407954,1.439853,1382.631572,2451.255445,694.515176
4,Boise City,98.0,Idaho,9.5,214237.0,POINT (-116.21461 43.61871),ID,Boise City-ID,Boise City,ID,...,Boise is a recreationalist's paradise. If you ...,7.1,7.2,7.1,0.386370,7.195404,2.419178,2588.190249,138.977594,413.363626


In [31]:
cities_joined["lakes_scaled"][0]

45096.02404867378

In [32]:
data_rivers = []
data_lakes = []
data_coasts = []

for i in range(len(x_es)):
    data_rivers.append([y_es[i],x_es[i],cities_joined["rivers_scaled"][i]])
    data_lakes.append([y_es[i],x_es[i],cities_joined["lakes_scaled"][i]])
    data_coasts.append([y_es[i],x_es[i],cities_joined["coasts_scaled"][i]])

In [33]:
cities_joined.head()

,city,rank,state,growth_from_2000_to_2013,population,geometry,state_code,city_key,City,State,...,Text,Overall Score,Quality of Life,Value,min_dist_to_rivers,min_dist_to_coasts,min_dist_to_lakes,rivers_scaled,coasts_scaled,lakes_scaled
0,Cleveland,48.0,Ohio,-18.1,390113.0,POINT (-81.69436 41.49932),OH,Cleveland-OH,Cleveland,OH,...,Once a haven for immigrants and a powerhouse o...,6.3,6.1,7.4,1.385577,5.297741,0.022175,721.721102,188.759703,45096.024049
1,Worcester,129.0,Massachusetts,5.8,182544.0,POINT (-71.80229 42.26259),MA,Worcester-MA,Worcester,MA,...,A vibrant and diverse community nicknamed The ...,6.5,7.0,6.5,1.962580,0.623077,1.373716,509.533275,1604.939242,727.952364
2,Columbia,192.0,South Carolina,11.7,133358.0,POINT (-81.03481 34.00071),SC,Columbia-SC,Columbia,SC,...,"Columbia, South Carolina, is a traditionally S...",6.5,6.1,7.2,1.057882,1.538227,6.822463,945.285376,650.099250,146.574634
3,Lafayette,214.0,Louisiana,11.0,124276.0,POINT (-92.01984 30.22409),LA,Lafayette-LA,Lafayette,LA,...,"In 2014, Lafayette was named the happiest city...",6.2,5.4,8.1,0.723258,0.407954,1.439853,1382.631572,2451.255445,694.515176
4,Boise City,98.0,Idaho,9.5,214237.0,POINT (-116.21461 43.61871),ID,Boise City-ID,Boise City,ID,...,Boise is a recreationalist's paradise. If you ...,7.1,7.2,7.1,0.386370,7.195404,2.419178,2588.190249,138.977594,413.363626


### Plot Map

In [34]:
#Map:
m = Map(center = (38,-90), zoom = 4, basemap = basemaps.Esri.WorldTopoMap, scroll_wheel_zoom = True)

#Screen Controls:
full_screen_control = FullScreenControl(position = "topright")
m.add_control(full_screen_control)
m.add_control(ScaleControl(position='bottomleft'))
m.add_control(LayersControl(position = "bottomright"))

draw_control = DrawControl()
draw_control.polyline =  {
    "shapeOptions": {
        "color": "#66ff33",
        "weight": 8,
        "opacity": 1.0
    }
}
draw_control.polygon = {
    "shapeOptions": {
        "fillColor": "#66ff33",
        "color": "#66ff33",
        "fillOpacity": 1.0
    },
    "drawError": {
        "color": "#ff0000",
        "message": "Oops!"
    },
    "allowIntersection": False
}
m.add_control(draw_control)


cities_joined_data = GeoData(geo_dataframe = cities_joined,
                   style = {'color': '#ff0000', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   name = '125 "Best" Cities')
m.add_layer(cities_joined_data)

rivers_data = GeoData(geo_dataframe = rivers,
                   style = {'color': '#b3d1ff', 'opacity':3, 'weight':5, 'dashArray':'2', 'fillOpacity':0.6},
                   name = 'Rivers')
m.add_layer(rivers_data)

lakes_data = GeoData(geo_dataframe = lakes,
                   style = {'color': '#b3d1ff', 'opacity':3, 'weight':2, 'dashArray':'2', 'fillOpacity':0.6},
                   name = 'Lakes')
m.add_layer(lakes_data)

airport_data = GeoData(geo_dataframe = airport,
                   style = {'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Cities')
#m.add_layer(airport_data)

coastline_data = GeoData(geo_dataframe = coastline,
                   style = {'color': '#0066ff', 'fillColor': '#b3d1ff', 'opacity':0.5, 'weight':10, 'dashArray':'2', 'fillOpacity':0.6},
                   name = 'Coastline')
m.add_layer(coastline_data)


m

Map(center=[38, -90], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

Here we can see the rivers, lakes, and coasts layers as well as the top places to live in the US!

#### And now, widgets!

In [35]:
# https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html
from IPython.display import display
import ipywidgets as widgets


from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


rheat = Heatmap(locations=data_rivers)
rheat.name = 'Rivers Heat map'
rheat.radius = 50
rheat.blur = 50
rheat.max = 0.0001
rheat.max = .5
#rheat.max_zoom = 4
rheat.gradient = {0.2: '#ffeee6', 0.4: '#ffddcc', 0.6: '#ffccb3', 0.8: '#ffbb99', 1.0: '#ffaa80'}
m.add_layer(rheat)



lheat = Heatmap(locations=data_lakes)
lheat.name = 'Lakes Heat Map'
lheat.radius = 50
lheat.blur = 50
lheat.max = 0.0001
lheat.max = .5
#lheat.max_zoom = 4
lheat.gradient = {0.2: '#ffeee6', 0.4: '#ffddcc', 0.6: '#ffccb3', 0.8: '#ffbb99', 1.0: '#ffaa80'}
m.add_layer(lheat)


cheat = Heatmap(locations=data_coasts)
cheat.name = 'Coasts Heat Map'
cheat.radius = 50
cheat.blur = 50
cheat.max = 0.0001
cheat.max = .5
#cheat.max_zoom = 4
cheat.gradient = {0.2: '#ffeee6', 0.4: '#ffddcc', 0.6: '#ffccb3', 0.8: '#ffbb99', 1.0: '#ffaa80'}
m.add_layer(cheat)

def f(Smoothing):
    if Smoothing == 0:
        Smoothing = 0.0001
    rheat.blur = Smoothing
    lheat.blur = Smoothing
    cheat.blur = Smoothing
b = interactive(f, Smoothing=widgets.IntSlider(min=5, max=100, step=5, value=50));




def f(Rivers):
    rheat.radius = Rivers
R = interactive(f, Rivers=widgets.IntSlider(min=5, max=100, step=5, value=50));

def f(Lakes):
    lheat.radius = Lakes        
L = interactive(f, Lakes=widgets.IntSlider(min=5, max=100, step=5, value=50));

def f(Coasts):
    cheat.radius = Coasts    
C = interactive(f, Coasts=widgets.IntSlider(min=5, max=100, step=5, value=50));


ui = widgets.HBox([b])
features = widgets.HBox([R, L, C])

display(ui, features)

m

Map(center=[38, -90], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

We can mix and max layers, combined with slider values to emphasize each city's proximity to rivers, lakes, and coasts! The smoothing slider effects how drastic the sensitivity is! Orange represents high proximity to desired features.